In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR']='/content'
!chmod 600 /content/kaggle.json
!kaggle datasets download -d tawsifurrahman/covid19-radiography-database
!unzip \*.zip && rm *.zip

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import glob
import cv2
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier

In [2]:
def prepare_input(filename):
  images = glob.glob(filename)
  imgs = np.array([cv2.resize(cv2.imread(file),(227,227)) for file in images])
  imgs=imgs[:,:,:,:1]
  imgs = np.array([arr.flatten() for arr in imgs])
  return imgs

In [3]:
covid_imgs = prepare_input("/content/COVID-19_Radiography_Dataset/COVID/*.png")
normal_imgs = prepare_input("/content/COVID-19_Radiography_Dataset/Normal/*.png")
pneum_imgs = prepare_input("/content/COVID-19_Radiography_Dataset/Viral Pneumonia/*.png")
print(covid_imgs.shape)
print(normal_imgs.shape)
print(pneum_imgs.shape)

(3616, 51529)
(10192, 51529)
(1345, 51529)


In [4]:
full_data = np.vstack((normal_imgs,covid_imgs,pneum_imgs))
print(full_data.shape)

(15153, 51529)


In [5]:
y_data = np.array([0]*len(normal_imgs) + [1]*len(covid_imgs) + [2]*len(pneum_imgs))
print(y_data.shape)

(15153,)


In [6]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=36)
for train_index, test_index in sss.split(full_data, y_data):
 X_train = full_data[train_index]
 y_train= y_data[train_index]
 X_test = full_data[test_index]
 y_test= y_data[test_index]

**Random-Forest Classifier**

In [ ]:
classifier = RandomForestClassifier(criterion='entropy',n_estimators=200)
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)
acc_RF = accuracy_score(y_test, y_pred)
print(acc_RF)

0.9224678323985483


**One V/s All Stochastic Gradient Descent**

In [ ]:
sgd_classifier = SGDClassifier(random_state=42)
sgd_classifier.fit(X_train,y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=42, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
sgd_pred = sgd_classifier.predict(X_test)
sgd_acc = accuracy_score(y_test, sgd_pred)
print(sgd_acc)

0.8383371824480369


**One V/s One Stochastic Gradient Descent**

In [7]:
from sklearn.multiclass import OneVsOneClassifier

In [10]:
ovo_sgd = OneVsOneClassifier(SGDClassifier(random_state=42))
ovo_sgd.fit(X_train,y_train)

OneVsOneClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                           class_weight=None,
                                           early_stopping=False, epsilon=0.1,
                                           eta0=0.0, fit_intercept=True,
                                           l1_ratio=0.15,
                                           learning_rate='optimal',
                                           loss='hinge', max_iter=1000,
                                           n_iter_no_change=5, n_jobs=None,
                                           penalty='l2', power_t=0.5,
                                           random_state=42, shuffle=True,
                                           tol=0.001, validation_fraction=0.1,
                                           verbose=0, warm_start=False),
                   n_jobs=None)

In [11]:
ovo_sgd_pred = ovo_sgd.predict(X_test)
ovo_sgd_acc = accuracy_score(y_test, ovo_sgd_pred)
print(ovo_sgd_acc)

0.7994061365885846
